In [218]:
!pip install openpyxl

# 1. Import libraries

In [219]:
# import common packages
import re
import sys
import itertools
import datetime
from tqdm.notebook import tqdm
import pandas_profiling
from datetime import datetime

# import visualization packages
import seaborn as sns
import matplotlib.pyplot as plt
import plotly.express as px

# import packages to work with the numeric, tabular data
import numpy as np 
import pandas as pd 

# import ML packages
from sklearn.preprocessing import LabelEncoder, OneHotEncoder, StandardScaler, PolynomialFeatures
from sklearn.feature_selection import f_regression, mutual_info_regression
from sklearn.model_selection import train_test_split, KFold, RandomizedSearchCV, cross_val_score
from sklearn.ensemble import RandomForestRegressor, BaggingRegressor, ExtraTreesRegressor, AdaBoostRegressor, GradientBoostingRegressor, StackingRegressor
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, make_scorer

from catboost import CatBoostRegressor
import xgboost as xgb

from hyperopt import tpe, hp, fmin, STATUS_OK,Trials
from hyperopt.pyll.base import scope

import warnings
warnings.filterwarnings("ignore")
import json
import datetime

In [220]:
!pip freeze > requirements.txt

In [221]:
# fix the RANDOM_SEED value so that the experiments are reproducible:
RANDOM_SEED = 42

In [222]:
# function to calculate MAPE metric
def mape(y_true, y_pred):
    return np.mean(np.abs((y_pred-y_true)/y_true))

# 2. Data Import and First Glance

In [223]:
# As of 11th of November 2021 there are 74K adds on auto ru in Moscow
train = pd.read_excel('../input/auto-ru-parsed-moscow/auto_ru_data_11_22_2022.xlsx') 
test = pd.read_csv('../input/sf-dst-car-price-prediction/test.csv')
sample_submission = pd.read_csv('../input/sf-dst-car-price-prediction/sample_submission.csv')

In [224]:
test

In [225]:
train = train.rename(columns={'sell_id': 'url_id', 'id': 'sell_id'})
train.head(2)

In [226]:
# lets drop first column as it was created by reseting index
# Lets also drop dobyType as it is the same as body_type but in russian
train = train.drop(['Unnamed: 0', 'bodyType'], axis=1)

In [227]:
test.head(2)

In [228]:
# Lets rename columns to the same as in train dataset
rename_columns ={
    "Владельцы": "owner",
    "Владение": "ownership",
    "ПТС": "PTS",
    "Привод": "gear",
    "Руль": "helm",
    "Состояние": "condition",
    "Таможня": "customs",
    "bodyType": "body_type"
}
test = test.rename(columns=rename_columns)
test.head(2)

In [229]:
# Lets compare the columns in two datasets
set(train.columns).difference(test.columns)
# Purpose group & price_segment & used_new & warranty are missing from test dataset
# Price is also missing because it is test dataset

In [230]:
train.info()
# As we can see there are in total 76K car adds aprsed from auto.ru in Moscow.
# It seems that there are a lot ot missing values, but actually all NA should represent 0 in dataset\
# Like for warranty if it is NA it means there is no warranty for this car

In [231]:
test.info()
# And 34K in test dataset

# 3. Data preprocessing

In [232]:
# Firstly lets fill all NA if possbile, if no drop a row or column
# Through the loop lets fill the missing price values with average based on vehicle model and year

for index, row in train[train['price'].isna()].iterrows():
    avg_price = train[(train['url_data1'] == row['url_data1']) & (train['url_data2'] == row['url_data2']) & (train['productionDate'] == row['productionDate'])]['price'].mean()
    train['price'][index] = avg_price
# And drop those where price is NA and model unique in dataset
train = train[train['price'].notna()]

In [233]:
# Based on the below i can assume that PTS is original for all new cars and there is only one used car
# with no info about PTS. I will assign ORIGINAL to all missing values
print(train[train['PTS'].isna()].value_counts(['used_new']),"\n_________SPLIT_______")
train.value_counts(['PTS', 'used_new'])

In [234]:
train['PTS'] = train['PTS'].fillna('ORIGINAL')
test['PTS'] = test['PTS'].fillna('ORIGINAL')

In [235]:
# Onwership NA should be reaplced with 0 since it might mean that there were no previous owner
# Ownership is the same as owner since there is an error in parsing script. Anyway, the truth value
# of ownership does not bring any value. Therefore, i will drop ownership from dataset
# The same as with owner data, the NA might mean there is no warranty for this car, lets replace it with 0
for column in ['ownership', 'owner', 'warranty']:
    print(train[column].value_counts())
train = train.drop('ownership', axis=1)
train['owner'] = train['owner'].fillna(0)
train['warranty'] = train['warranty'].fillna(0)
train['warranty'] = train['warranty'].replace(bool(1), 1)


In [236]:
# The last column with NA is priceCurrency, i am quite sure that the prices are mentiond in RUR
train['priceCurrency'] = train['priceCurrency'].fillna('RUR')

In [237]:
# for column in train.columns:
#     print(column, '\n', train[column].unique(), '\n')
    
# sell_id - is id of add
# brand - according to json response while parsin data it is human readable format :)
#         drop this column, since we have it in url_data1
# color - is a HEX color, we will convert it to readable format and later check if it affects on the price
# complectation_dict - has the same values as equipment dict. Drop the column
# description - is a text to the add. We might get some common words that might help to bargue
# engineDisplacement - drop we have this info in separate columns GEAR, ENGINE POWER, vehicleTransmission
# enginePower - is the power of engine
# equipment_dict - can help to add new features to dataset.
# fuelType - type of fuel
# mileage - mileage of car
# modelDate - a year when the car was produced
# model_info - drop it is the same as url_data2
# model_name - drop it is the same as url_data2
# name - drop we have this info in separate columns GEAR, ENGINE POWER, vehicleTransmission
# numberOfDoors - number of doors
# priceCurrency - currecny, but i think it does not bring any additional value, since all prices are in RUR
# productionDate - No idea what it is, drop
# url_id - id for url
# super_gen - can drop this column since we have this data in separate columns
# vehicleConfiguration - vehicleConfiguration consists of the same data we have already in dataset
# vehicleTransmission - is a gearbox of a car. Not so many options
# vendor - 
# owner - how many onwers a car has
# PTS - orginal documents or duplicated
# gear - rear wheel drive or front wheel drive or 4WD
# helm - right side steering wheel or left side
# condition - whether a car needs some maintanance or no. All values are set to True. We can drop it
# customs - custom clearance
# price - our target column
# warranty - if a car has warranty or no
# used_new - new car or old
# body_type - type of a car SUV or sedan
# price_segment - premium, economy or business
# purpose_group - for familly, business or city
# url_data1 - brand
# url_data2 - model
# url_data3 - url id

In [238]:
columns_to_drop = ['brand',
'complectation_dict',
'engineDisplacement',
'model_info',
'model_name',
'name',
'priceCurrency',
'modelDate',
'url_id',
'super_gen',
'vehicleConfiguration',
'condition',
'url_data3',
'warranty',
'used_new',
'price_segment',
'purpose_group']
train = train.drop(columns_to_drop, axis=1)

In [239]:
rename_train = {
    'url_data1': 'brand',
    'url_data2': 'model_name'
}
train = train.rename(columns=rename_train)

In [240]:
# Lets do the same for test dataset
# for column in test.columns:
#     print(column, '\n', test[column].unique(), '\n')
    
# body_type - drop
# brand - the same as url_data1
# car_url - drop
# color - the same as color
# complectation_dict - drop all info in equipment_dict
# description - OK
# engineDisplacement - we have it in supergen
# enginePower - we have it in supergen
# equipment_dict - we will extract info
# fuelType - OK
# image - drop
# mileage - OK
# modelDate - drop
# model_info - drop
# model_name - OK
# name - drop
# numberOfDoors - OK
# parsing_unixtime - drop
# priceCurrency - drop
# productionDate - OK
# sell_id - OK
# super_gen - OK
# vehicleConfiguration - ok
# vehicleTransmission - drop
# vendor - ok
# owner - ok
# ownership - drop
# PTS - ok
# gear - drop
# helm - ok
# condition - drop
# customs - ok

In [241]:
drop_columns_test = [
'body_type',
'car_url',
'complectation_dict',
'engineDisplacement',
'enginePower',
'image',
'modelDate',
'model_info',
'name',
'parsing_unixtime',
'priceCurrency',
'vehicleTransmission',
'ownership',
'gear',
'condition'
]
test = test.drop(drop_columns_test, axis=1)

In [242]:
test = test.rename(columns={'vehicleConfiguration': 'body_type'})

In [243]:
test['enginePower'] = test['super_gen'].apply(lambda x: json.loads(x)['power'])
test['gear'] = test['super_gen'].apply(lambda x: json.loads(x)['gear_type'])
test['vehicleTransmission'] = test['super_gen'].apply(lambda x: json.loads(x)['transmission'])

In [244]:
test = test[['sell_id',
 'color',
 'description',
 'enginePower',
 'equipment_dict',
 'fuelType',
 'mileage',
 'numberOfDoors',
 'productionDate',
 'vehicleTransmission',
 'vendor',
 'owner',
 'PTS',
 'gear',
 'helm',
 'customs',
 'body_type',
 'brand',
 'model_name']]

In [245]:
# Now we can see that columns are identical in both datasets
set(train.columns).difference(test.columns)

In [246]:
train.head(2)

In [247]:
# Before merging two datasets together, lets adjust prices from parsed datset to 2020 prices, becuse the test dataset was prepared in 2020

![](https://pbs.twimg.com/media/E81-YJwX0AYIT5C.jpg)

* According to the above graph, the prices increased by around 17% which means that we need to adjust prices by multiplying each on 0.83

In [248]:
train['price'] = train['price'].apply(lambda x: x*0.83)

In [249]:
# Lets also create ID for test and train datasets and merge these two into one
# A Dummy column price is also created for test dataset
train['sample'] = 0
test['sample'] = 1
test['price'] = 1

df = train.append(test, sort=False).reset_index(drop=True)

In [250]:
df = df.drop('index', axis=1)

# Now our dataframes are identical, and lets categorize the values to the same format

In [251]:
color = {
    'FAFBFB': 'white', 
    '97948F': 'gray', 
    '4A2197': 'blue', 
    '040001': 'black', 
    '007F00': 'green', 
    '660099': 'purple',
    '0000CC': 'blue', 
    'EE1D19': 'red', 
    'FFD600': 'yellow', 
    'C49648': 'brown', 
    '22A0F8': 'blue', 
    'CACECB': 'gray',
    '200204': 'black',
    'DEA522': 'yellow', 
    'FF8649': 'orange',
    'FFC0CB': 'pink', 
    'синий': 'blue',
    'чёрный': 'black', 
    'серый': 'gray',
    'коричневый': 'brown', 
    'белый': 'white', 
    'пурпурный': 'purple', 
    'бежевый': 'beige', 
    'серебристый': 'silver',
    'красный': 'red', 
    'зелёный': 'green', 
    'жёлтый': 'yellow',
    'голубой': 'blue', 
    'оранжевый': 'orange',
    'фиолетовый': 'purple', 
    'золотистый': 'gold',
    'розовый': 'pink'
}

fuelType = {'бензин': 'GASOLINE',
            'дизель': 'DIESEL',
            'гибрид': 'HYBRID',
            'электро': 'ELECTRO',
            'газ': 'LPG'}
owner = {'3 или более': 3,
         '1\xa0владелец': 1,
         '2\xa0владельца': 2}
PTS = {'Оригинал': 'ORIGINAL',
       'Дубликат': 'DUPLICATE'}
helm = {'Левый': 'LEFT',
        'Правый': 'RIGHT'}
customs = {'Растаможен': True}

In [252]:
df['color'] = df['color'].apply(lambda x: color[x])
df['fuelType'] = df['fuelType'].apply(lambda x: fuelType[x] if x in fuelType.keys() else x)
df['owner'] = df['owner'].apply(lambda x: owner[x] if x in owner.keys() else x)
df['PTS'] = df['PTS'].apply(lambda x: PTS[x] if x in PTS.keys() else x)
df['helm'] = df['helm'].apply(lambda x: helm[x] if x in helm.keys() else x)
df['customs'] = df['customs'].apply(lambda x: customs[x] if x in customs.keys() else x)

In [253]:
# Lets have a look at all options we have in dataset
options = []
for option_list in df['equipment_dict']:
    if type(option_list) == type(float(0.1)):
        continue
    elif len(option_list) == 2:
        continue
    else:
        options_all = json.loads(option_list.replace("'", '"').replace("True", '"True"')).keys()
        for option in options_all:
            options.append(option)
print("Total LEN of all options is: ",len(set(options)))

In [254]:
# Since it is a big number to add as separate columns, i decided to create a column with len of cars options
# The number will represnt an amount of options that car has
options1 = []
for option_list1 in df['equipment_dict']:
    if type(option_list1) == type(float(0.1)):
        options1.append(0)
    elif len(option_list1) == 2:
        options1.append(0)
    else:
        options_all1 = json.loads(option_list1.replace("'", '"').replace("True", '"True"')).keys()
        options1.append(len(options_all1))
df['options'] = options1
df = df.drop('equipment_dict', axis=1)

In [255]:
# body_type has number of doors, this needs to be removed since we have this info in a separate column. Additionally, in the test DF there are some additional data included in body_type
body_type_new = []
for i in df['body_type']:
    if "_" in i:
        body_type_new.append(i[:i.find("_")])
    elif " " in i:
        body_type_new.append(i[:i.find(" ")])
    else:
        body_type_new.append(i)
df['body_type'] = body_type_new

In [256]:
print(df['customs'].value_counts())
# We dont have any other value than True, i see no point keeping it here since all cars got cleared its customs
df = df.drop('customs', axis=1)

In [257]:
words_list = {"торг": 0, "скидка": 0, "срочно": 0}
for words in df['description']:
    for key_word in words_list:
        if str(words) == 'nan':
            continue
        if key_word in words.lower():
            words_list[key_word] += 1
print(words_list)

In [258]:
# As we can see from the above, all these three key words has potential impact on the price.
# A column will be created to count these words in description if one match it is 1 if two then two and so on
words_list = ["торг", "скидка", "срочно"]
key_words = []
for words in df['description']:
    counter = 0
    for key_word in words_list:
        if str(words) == 'nan':
            continue
        if key_word in words.lower():
            counter += 1
    key_words.append(counter)
df['key_words'] = key_words
df = df.drop('description', axis=1)

# 4. EDA

In [259]:
# First lets drop unneccesasry columns
df = df.drop('sell_id', axis=1)

In [260]:
pandas_profiling.ProfileReport(df)

Dataset consist of 20 objects and 108102 records;
There are no NAN values as well as no duplicates;
There is a high coorelation between productionDate and owner and milaege

In [261]:
# lets assign column names to lists for easier refference in case numerical, categorial etc
num_cols = ['enginePower', 'mileage', 'productionDate', 'options', 'key_words']
cat_cols = ['color', 'fuelType', 'numberOfDoors', 'vehicleTransmission', 'vendor', 'owner', 'gear', 'body_type', 'brand', 'model_name']
bin_cols = ['helm', 'PTS']
target_col = ['price']
help_col = ['sample']
all_cols = num_cols + cat_cols + bin_cols + target_col + help_col

In [262]:
# Analysis of numerical columns, abnormal values, correlation, distribution
sns.pairplot(df[num_cols])

There is no normal distribution it is eatiher skewed to the lft or right. Log values needs to be checked in terms of distribution
High correlation between variables does not exist here

In [263]:
# Statistical data confirms that the variables are skewed, however, we can see that key_words column has relatively low std.
df[num_cols].describe()

In [264]:
# построим сравнительные графики без логарифмирования и после логарифмирования
data = pd.DataFrame()
data['price']=df['price']

for item in num_cols:
    plt.suptitle('LOG')
    fig, ax =plt.subplots(1,3, figsize=(18,4))
    sns.distplot(df[item].dropna(), kde = False, rug=False, ax=ax[0])
    sns.boxplot(df[item], ax=ax[1])
    df.plot(x = item, y = 'price', kind = 'scatter',color="0.5", ax=ax[2])
    fig.show()
    
    data[item] = df[item].apply(lambda w: np.log(w + 1))
    fig, ax =plt.subplots(1,3, figsize=(18,4))
    sns.distplot(data[item].dropna(), kde = False, rug=False, ax=ax[0])
    sns.boxplot(data[item], ax=ax[1])
    data.plot(x = item, y = 'price', kind = 'scatter',color="0.5", ax=ax[2])
    fig.show()

Log values are good for engine power and milage, log vlaues of production date does not change anything. 
Lets only convert these two to log

In [265]:
# Correlation of numerical and target variables
plt.figure(figsize=(15, 8))
sns.heatmap(df[num_cols + target_col].corr(), annot = True, cmap = 'coolwarm');

* There are no high coorelated variables, i would not drop any of these.
* Also, we can see enginePower is quite coorelated with our target variable price. Which makes sense, since a car getting younger more options it gets.
* Negative coorelation of milage with our target variable tells us that as a car getting older the price drops. However, it is not fully true because the value is only -0.59

In [266]:
df.query('sample == 1').enginePower.describe(), df.query('sample == 0').enginePower.describe()

* It is interesting to see a car with 1020 HP and a car with 5 HP in our parsed dataset.
* Our test dataset seems okay, since the min and max are quite reasonable

In [267]:
print(df[df['enginePower'] > 1000].value_counts(['brand', 'model_name']))
# All 13 cars with 1020 HP are Teslas
# I checked the offical website of Tesla, and it is True that new Tesla model X has 1020 HP. 
# I will leave variable this just for now

In [268]:
print(df[df['enginePower'] < 10].value_counts(['brand', 'model_name', 'enginePower']))
# According to technical info of this specific car, that is also true, that a car might has 5 HP only.

# Lets check now our categorial variables

In [269]:
for item in bin_cols + cat_cols:  
    if item not in ['model_name']: 
        # слишком много значений в этих колонках перегрузят график, сделав его не информативным
        fig, ax =plt.subplots(1,2, figsize=(25,4))
        sns.countplot(x = item, data = df, ax=ax[0])
        # для наглядности анализа распределения цены для значений 
        # признака уберём хвосты задав quantile(0.85)
        sns.boxplot(x=item, y='price', data=df[df['price'] <= df['price'].quantile(0.85)], ax=ax[1])
        fig.show()

In general there are no balanced variables
* helm - right is the most common in the dataset and usually more expensive that left hand side
* PTS - original is the  most common and more expensive than duplicate
* color - grey, black and white cars are the most popular however, i do not see any coorelation between price and color. But we can crete a new columns saying that car has one of the most popular color or no 
* fuelType - hubrid cars are expnsive, the cheapest option comes with benzin 
* numberOfDoors - cars with 0 doors are the most expensive in dataset, the majority of cars are with 5 doors
* vehicleTransmission - cars with automatic gear are cheap in general
* vendor - Chiniease vendors usually are no cheap
* owner - it is obvious that cars with no owners in the past have a higher price than those with an owner or few. Genrally saying more owners car has less seling price is expected
* gear - the most popular is FWD then AWD, price is higher for AWD 
* body_type - the most poular body types are sedan and allroad (we can create a few new columns saying whether it is a popualr body type or rare)
* brand - there are a lot of brands, will look at this a bit closer later

In [270]:
plot = df.copy()
segment = []
for price in plot['price']:
    if price < 1000000:
        segment.append(1)
    elif price < 2000000:
        segment.append(2)
    elif price < 3000000:
        segment.append(3)
    elif price < 4000000:
        segment.append(4)
    elif price < 5000000:
        segment.append(5)
    elif price < 6000000:
        segment.append(6)
    elif price < 7000000:
        segment.append(7)
    elif price < 8000000:
        segment.append(8)
    elif price < 9000000:
        segment.append(9)
    elif price < 10000000:
        segment.append(10)
    elif price >= 10000000:
        segment.append(11)
plot['segment'] = segment
fig = px.treemap(plot,
                path=['segment', 'brand', 'model_name'],
                height=750)
fig

As we can see, the majority of the cars within 1 million around 50%. and around 75% within 2 million

In [271]:
df[df['sample'] == 0]['price'].hist(figsize=(10, 4))
# Distribution is not normal lets try log values

In [272]:
np.log2(df[df['sample'] == 0]['price']).hist(figsize=(10, 4))
# Now the distribution is close to normal 

# Feature Engeenering

* rare body - if a body type is a not popular
* popular body - if a body type is popular
* popular color - if color is popular
* rare color - if color is not popular
* new used - whether the car is used or new
* age car - age of a car
* mileage year - mileage per year

In [273]:
df['rare_body'] = df['body_type'].apply(lambda x: 1 if x in ['SPEEDSTER', 'TARGA', 'FASTBACK', 'LIMOUSINE', 'MICROVAN'] else 0)
df['popular_body'] = df['body_type'].apply(lambda x: 1 if x in ['SEDAN', 'ALLROAD'] else 0)
df['popular_color'] = df['color'].apply(lambda x: 1 if x in ['grey', 'black', 'white'] else 0)
df['rare_color'] = df['color'].apply(lambda x: 1 if x in ['orange', 'pink', 'beige', 'gold', 'purpule', 'yellow'] else 0)
df['new_used'] = df['mileage'].apply(lambda x: 'new' if x < 5000 else 'used')
df['age_car'] = 2021 - df['productionDate'].replace(2022, 2021)
df['mileage_year'] = df['mileage']/df['age_car']
new_col = ['rare_body', 'popular_body', 'popular_color', 'rare_color', 'new_used', 'age_car', 'mileage_year']

In [274]:
df['log_price'] = np.log2(df['price']+1)
df['log_mileage'] = np.log2(df['mileage']+1)
df['log_enginePower'] = np.log2(df['enginePower']+1)

In [275]:
df['mileage_year'] = df['mileage_year'].replace(df['mileage_year'].max(), 0)
df['mileage_year'] = df['mileage_year'].fillna(0)

In [276]:
for item in ['age_car', 'mileage_year']:
    plt.suptitle('Логарифмируем')
    fig, ax = plt.subplots(1, 3, figsize=(18, 4))
    sns.distplot(df[item], kde=False, rug=False, ax=ax[0])
    sns.boxplot(df[item], ax=ax[1])
    df.plot(x=item, y='price', kind='scatter', color="0.5", ax=ax[2])
    fig.show()

    data[item] = df[item].replace(-1, 0).apply(lambda w: np.log(w + 1))
    fig, ax = plt.subplots(1, 3, figsize=(18, 4))
    sns.distplot(data[item].dropna(), kde=False, rug=False, ax=ax[0])
    sns.boxplot(data[item], ax=ax[1])
    data.plot(x=item, y='price', kind='scatter', color="0.5", ax=ax[2])

In [277]:
df.info()

In [278]:
df['age_car'] = np.log2(df['age_car']+1)
df['mileage_year'] = np.log2(df['mileage_year']+1)

In [279]:
# Correlation of numerical and target variables
plt.figure(figsize=(15, 8))
sns.heatmap(df[['rare_body', 'popular_body', 'popular_color', 'rare_color', 'age_car', 'mileage_year', 'price']].corr(), annot = True, cmap = 'coolwarm');

* age car has a negative correlation which tells us that higher age car lower price and quite the same with mileage year

In [280]:
for item in ['rare_body', 'popular_body', 'popular_color', 'rare_color', 'new_used']:
    fig, ax = plt.subplots(1, 2, figsize=(15, 4))
    sns.countplot(x=item, data=df, ax=ax[0])
    # для наглядности анализа распределения цены для значений признака уберём хвосты
    # задав quantile(0.85)
    sns.boxplot(x=item, y='price',
                data=df[df['price'] <= df['price'].quantile(0.85)], ax=ax[1])
    fig.show()

In [281]:
num_cols = ['enginePower', 'mileage', 'options', 'key_words'] + ['age_car', 'mileage_year']
cat_cols = cat_cols + ['new_used']
bin_cols = bin_cols + ['rare_body', 'popular_body', 'popular_color', 'rare_color']

In [282]:
df[df['sample'] == 0].info()

In [283]:
df.info()

In [284]:
imp_num = pd.Series(f_regression(
    df[df['sample'] == 0][num_cols], df[df['sample'] == 0]['price'])[0], index=num_cols)
imp_num.sort_values(inplace=True)
imp_num.plot(kind='barh')
plt.xlabel('f Statistic')

* the most valuable variable is enginePower

In [285]:
for colum in ['helm', 'vehicleTransmission', 'brand', 'PTS']:
    df[colum] = df[colum].astype('category').cat.codes

cols_to_encode = list(set(df.columns) & set(cat_cols))
for colum in cols_to_encode:
    df[colum] = df[colum].astype('category').cat.codes

In [286]:
imp_cat = pd.Series(
    mutual_info_regression(
        df[df['sample'] == 1][list(set(df.columns) & set(cat_cols + bin_cols))], 
        df[df['sample'] == 1]['price'], 
        discrete_features=True), index=list(set(df.columns) & set(cat_cols+bin_cols))
)
imp_cat.sort_values(inplace=True)
imp_cat.plot(kind='barh', title='Cat & Bin Cols')
plt.show()

* the most valuable is model_name

# Machine Learning

In [313]:
X = df[df['sample'] == 0].drop(['price', 'sample', 'log_price', 'mileage', 'enginePower', 'color', 'productionDate'], axis=1)

X_t = df[df['sample'] == 1].drop(['price', 'sample', 'log_price', 'mileage', 'enginePower', 'color', 'productionDate'], axis=1)

y = df[df['sample'] == 0]['price']

scaler = StandardScaler()

X = scaler.fit_transform(X)

X_t = scaler.fit_transform(X_t)

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, shuffle=True, random_state=RANDOM_SEED)

# CatBoost

### Fit

In [314]:
cat_boost = CatBoostRegressor(iterations=5000, random_seed=RANDOM_SEED, eval_metric='MAPE',
                              custom_metric=['R2', 'MAE'], silent=True,)
cat_boost.fit(X_train, y_train, eval_set=(X_test, y_test),
              verbose_eval=0, use_best_model=True, plot=True)
y_pred = cat_boost.predict(X_test)
print(f"The MAPE mertics of the CatBoostRegressor model is: {(mape(y_test, y_pred))*100:0.2f}%")

### Target Log

In [315]:
cat_boost_log = CatBoostRegressor(iterations = 5000, random_seed = RANDOM_SEED, eval_metric='MAPE',
                              custom_metric=['R2', 'MAE'], silent=True)
cat_boost_log.fit(X_train, np.log(y_train), eval_set=(X_test, np.log(y_test)),
              verbose_eval=0, use_best_model=True)
predict_test = np.exp(cat_boost_log.predict(X_test))
print(f"The MAPE mertics of the CatBoostRegressor LOG model is: {(mape(y_test, predict_test))*100:0.2f}%")

# Random Forest

### Fit

In [316]:
rf = RandomForestRegressor(random_state=RANDOM_SEED, n_jobs=-1, verbose=1)
rf.fit(X_train, y_train)
predict_rf = rf.predict(X_test)

print(f"The MAPE mertics of the Random Forest model is: {(mape(y_test, predict_rf) * 100):0.2f}%.")

### Target Log

In [317]:
rf_log = RandomForestRegressor(random_state=RANDOM_SEED, n_jobs=-1, verbose=1)
rf_log.fit(X_train, np.log(y_train))
predict_rf_log = np.exp(rf_log.predict(X_test))

print(f"The MAPE mertic for the Random Forest LOG model is : {(mape(y_test, predict_rf_log) * 100):0.2f}%.")

# ExtraTreesRegressor

### Target Log

In [318]:
etr_log = ExtraTreesRegressor(random_state=RANDOM_SEED, n_jobs=-1, verbose=1)
etr_log.fit(X_train, np.log(y_train))
y_pred = np.exp(etr_log.predict(X_test))
etr_log_mape_value = mape(y_test, y_pred)
print(f"The MAPE mertic for the default ExtraTreesRegressor LOG is: {(np.mean(etr_log_mape_value) * 100):0.2f}%.")

# XGBoostRegressor

### Target Log

In [319]:
xgb_log = xgb.XGBRegressor(objective='reg:squarederror', colsample_bytree=0.5, learning_rate=0.05, max_depth=12, alpha=1,
                           n_estimators=1000, random_state=RANDOM_SEED, verbose=1, n_jobs=-1)
xgb_log.fit(X_train, np.log(y_train))
y_pred = np.exp(xgb_log.predict(X_test))
xgb_log_mape_value = mape(y_test, y_pred)
print(f"The MAPE mertic for the XGBRegressor LOG is: {(np.mean(xgb_log_mape_value) * 100):0.2f}%.")

In [320]:
# # Cheking hyperparameters
# def objective(params):
#     model = xgb.XGBRegressor(
#         learning_rate=params['learning_rate'],
#         max_depth=int(params['max_depth']),
#         n_estimators=int(params['n_estimators']),
#         random_state=RANDOM_SEED,
# #         verbose=0,
#         n_jobs=-1
#     )
#     model.fit(X_train, np.log(y_train))
#     pred=model.predict(X_test)
#     score=mape(y_test,np.exp(pred))
#     return score

# # Setting hyperparameters
# def optimize(trial):
#     params={
#         'n_estimators': hp.uniform('n_estimators',100,500),
#         'max_depth': hp.uniform('max_depth',5,35),
#         'learning_rate': hp.uniform('learning_rate',0.1,0.6),
#     }
#     best=fmin(fn=objective, space=params, algo=tpe.suggest, trials=trial, max_evals=100, rstate=np.random.RandomState(RANDOM_SEED))
#     return best

# trial=Trials()
# best=optimize(trial)
# print(best)

# 100%|██████████| 100/100 [42:19<00:00, 25.40s/trial, best loss: 0.15943050949980647]
# {'learning_rate': 0.1001854950773469, 'max_depth': 12.381663885775968, 'n_estimators': 168.86663413217474}

In [321]:
xgb_log = xgb.XGBRegressor(
    objective='reg:squarederror',
    colsample_bytree=0.5,
    learning_rate=0.1,
    max_depth=12,
    alpha=1,
    n_estimators=169,
    random_state=RANDOM_SEED,
    verbose=1,
    n_jobs=-1
)
xgb_log.fit(X_train, np.log(y_train))
y_pred = np.exp(xgb_log.predict(X_test))
xgb_log_mape_value = mape(y_test, y_pred)
print(f"The MAPE mertic for the XGBRegressor LOG is: {(np.mean(xgb_log_mape_value) * 100):0.2f}%.")

* Parameters do not affect the model to much, however, i applied a bit different learing rate and we got a bit better result.

In [322]:
xgb_log = xgb.XGBRegressor(
    objective='reg:squarederror',
    colsample_bytree=0.5,
    learning_rate=0.03,
    max_depth=12,
    alpha=1,
    n_estimators=1000,
    random_state=RANDOM_SEED,
    verbose=1,
    n_jobs=-1
)
xgb_log.fit(X_train, np.log(y_train))
y_pred = np.exp(xgb_log.predict(X_test))
xgb_log_mape_value = mape(y_test, y_pred)
print(f"The MAPE mertic for the XGBRegressor LOG is: {(np.mean(xgb_log_mape_value) * 100):0.2f}%.")

# Stacking

Lets try to combine two best models CatBoostRegressor and XGBRegressor with stacking that might help to improve our MAPE

In [323]:
estimators = [
    ('cbr', CatBoostRegressor(iterations=5000, random_seed=RANDOM_SEED, eval_metric='MAPE', silent=True)),
    ('xgb', xgb.XGBRegressor(objective='reg:squarederror', colsample_bytree=0.5, learning_rate=0.03, max_depth=12, alpha=1, n_jobs=-1, n_estimators=1000, random_state=RANDOM_SEED))
]

sr_log = StackingRegressor(
    estimators=estimators,
    final_estimator=LinearRegression()
)

# For training, fit() is used
sr_log.fit(X_train, np.log(y_train))

# For MAPE metric (or any other), we need the predictions of the model
y_pred = np.exp(sr_log.predict(X_test))

print(f"The MAPE mertic for the default StackingRegressor model is: {(mape(y_test, y_pred) * 100):0.2f}%.")

The best result shows XGBRegressor with log of target variable with MAPE result 15.25%. The second place is taken by CatBoostRegressor with result 15.85%. However, combining these two did not help to improve MAPE metric

# Conclucions

* The model can be improved by generating new features, optimizing hyperparameters and performing more in depth EDA.
* As for the submission the CatBoostRegressor is used to predict the target variable and upload a submission file to the Kaggle competition

In [325]:
predict_submission = np.exp(xgb_log.predict(X_t))
sample_submission['price'] = predict_submission
sample_submission.to_csv(f'submission_final.csv', index=False)
sample_submission.head(10)